<a href="https://colab.research.google.com/github/afvallejo/CSHL2022/blob/main/1_kallisto_alignment_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages

In [ ]:
!wget https://github.com/pachterlab/kallisto/releases/download/v0.46.1/kallisto_linux-v0.46.1.tar.gz

In [ ]:
!tar -xf kallisto_linux-v0.46.1.tar.gz

In [ ]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip

In [ ]:
!unzip fastqc_v0.11.9.zip

In [ ]:
!chmod +x FastQC/fastqc

In [ ]:
!pip install multiqc

# Run Kallisto

## Usage: kallisto quant [arguments] FASTQ-files


Required arguments:
-i, --index=STRING            Filename for the kallisto index to be used for
                              quantification
-o, --output-dir=STRING       Directory to write output to

Optional arguments:
    **--bias                    Perform sequence based bias correction**
**-b, --bootstrap-samples=INT   Number of bootstrap samples (default: 0)**
    --seed=INT                Seed for the bootstrap sampling (default: 42)
    --plaintext               Output plaintext instead of HDF5
    --fusion                  Search for fusions for Pizzly
    --single                  Quantify single-end reads
    --single-overhang         Include reads where unobserved rest of fragment is
                              predicted to lie outside a transcript
    **--fr-stranded             Strand specific reads, first read forward
    --rf-stranded             Strand specific reads, first read reverse**
-l, --fragment-length=DOUBLE  Estimated average fragment length
-s, --sd=DOUBLE               Estimated standard deviation of fragment length
                              (default: -l, -s values are estimated from paired
                               end data, but are required when using --single)
**-t, --threads=INT             Number of threads to use (default: 1)**
    --pseudobam               Save pseudoalignments to transcriptome to BAM file
    --genomebam               Project pseudoalignments to genome sorted BAM file
-g, --gtf                     GTF file for transcriptome information
                              (required for --genomebam)
-c, --chromosomes             Tab separated file with chromosome names and lengths
                              (optional for --genomebam, but recommended)

In [ ]:
!kallisto/kallisto

# Get the cDNA file

In [ ]:

!wget ftp://ftp.ensembl.org/pub/release-95/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz


# Create index

In [ ]:
!kallisto/kallisto index --make-unique -i kallisto_index.idx Homo_sapiens.GRCh38.cdna.all.fa.gz

## Downloading the data 


**Mycobacterium tuberculosis Invasion of the Human Lung**: First Contact.
Maertzdorf J1, Tönnies M2, Lozza L1, Schommer-Leitner S1, Mollenkopf H1, Bauer TT2, Kaufmann SHE1.

[Rawdata](https://www.ebi.ac.uk/ena/data/view/PRJNA447933)

In [ ]:
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR691/000/SRR6914400/SRR6914400_1.fastq.gz
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR691/000/SRR6914400/SRR6914400_2.fastq.gz

In [ ]:
ls

# QC

In [ ]:
!FastQC/fastqc SRR6914400_2.fastq.gz

# Run Kallisto

## un-stranded

In [ ]:
!kallisto/kallisto quant -i kallisto_index.idx -o Kallisto_results -t 2 SRR6914400_1.fastq.gz SRR6914400_2.fastq.gz

## stranded fr

In [ ]:
!kallisto/kallisto quant -i kallisto_index.idx --fr-stranded -o Kallisto_results -t 2 SRR6914400_1.fastq.gz SRR6914400_2.fastq.gz


## stranded rf

In [ ]:
!kallisto/kallisto quant -i kallisto_index.idx --rf-stranded -o Kallisto_results -t 2 SRR6914400_1.fastq.gz SRR6914400_2.fastq.gz

## Alignment

In [ ]:
!kallisto/kallisto quant -i kallisto_index.idx --bias -b 50 -o Kallisto_results -t 2 SRR6914400_1.fastq.gz SRR6914400_2.fastq.gz >>log 2>&1


# read the output

In [ ]:
import pandas as pd 
data = pd.read_csv("/content/Kallisto_results/abundance.tsv",sep='\t') 
data.sort_values(by=['tpm'],ascending=False)

# integrated QC

In [ ]:
!multiqc .

# About bias correction 

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('9xskajkNJwg')